In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


import numpy as np
import sys
sys.path.append('/Users/diegofiori/Desktop/epfl/master_thesis/master_thesis/')
from input_reader import read_simulation_file, get_all_time_ids
from matplotlib.animation import ArtistAnimation

import matplotlib.cm as cm
from tqdm.notebook import tqdm

In [2]:
path = '/Users/diegofiori/Desktop/epfl/master_thesis/Reverse/'
space_index, field = 0, 'temperature'
img = [] # some array of images
time_ids = get_all_time_ids(path)
for i in tqdm(range(0, len(time_ids), 6)):
    simulation_slices = read_simulation_file(path, field, time_ids[i:i+6])
    simulation_slices = [simulation_slices[j] for j in range(0, len(simulation_slices), 80)]
    img += simulation_slices

In [16]:
def create_video(list_of_images, time_series, interval=50, repeat=True, repeat_delay=100, **fig_params):
    flag = True
    fig = plt.figure(**fig_params)
    viewer_1 = fig.add_subplot(121)
    viewer_2 = fig.add_subplot(122)
    fig.show()
    plt.ion()
    while flag:
        try:
            for i, image in enumerate(list_of_images):
                viewer_1.clear()
                viewer_1.imshow(image)
                viewer_2.clear()
                viewer_2.plot(time_series)
                viewer_2.plot([i, i], [np.min(time_series), np.max(time_series)])
                plt.pause(interval/1000)
            if repeat:
                plt.pause(repeat_delay/1000)
            else:
                flag = False
        except KeyboardInterrupt:
            flag = False

In [14]:
time_series = np.random.randn(len(img))

In [18]:
create_video(img, time_series, interval=50, repeat_delay=1000)

TclError: can't invoke "update" command: application has been destroyed

## Load the important features

In [ ]:
topological_features = 